In [ ]:
from source_files_extended import *
from plot_helpers import *
from source_files import CASI_CLASSES, CASI_MAPPING

import query_helpers

import math
import pandas as pd

# Analysis

In [ ]:
aso_mask = load_aso_depth().mask
hillshade_snow_on, hillshade_snow_free = load_hillshade(aso_mask)

In [ ]:
df = pd.DataFrame({
        'sfm_snow_depth': load_sfm_depth(aso_mask).ravel(),
        'hillshade_snow_on': hillshade_snow_on.ravel(),
        'hillshade_snow_free': hillshade_snow_free.ravel(),
        'casi_class': load_classifier_data(aso_mask).ravel(),
})

df.dropna(inplace=True, how='any', subset=['sfm_snow_depth'])
df['casi_class'] = pd.cut(df['casi_class'], CASI_MAPPING, labels=CASI_CLASSES)

positive_sfm = query_helpers.get_positive(df, 'sfm_snow_depth')
negative_sfm = query_helpers.get_negative(df, 'sfm_snow_depth')

vegetation = ((negative_sfm.casi_class == 'Vegetation') | (negative_sfm.casi_class == 'Water'))
open_areas = ~vegetation

HILLSHADE_MIN = 0
HILLSHADE_MAX = 255

hillshade_range = np.arange(HILLSHADE_MIN, HILLSHADE_MAX + 1, 1)
min_depth = math.floor(negative_sfm.sfm_snow_depth.min())
bins = np.arange(min_depth, 0.5, 0.5)

In [ ]:
def compare_column(data, column, title):
    plot_data = data
    plot_group = plot_data.groupby(column)
    plot_data_median = plot_group.median()
    plot_data_count = plot_group.count()

    data = [
        plot_data[column],
        plot_data.sfm_snow_depth,
        plot_data_median.index, 
        plot_data_median.sfm_snow_depth,
        plot_data_count.index, 
        plot_data_count.sfm_snow_depth,
    ]
    
    fig, (ax1, ax2) = plt.subplots(
        2, 1, sharex=True, 
        figsize=(8,10), gridspec_kw={ 'hspace': 0.05}, 
    )

    ax1.set_facecolor('whitesmoke')
    ax1.set_facecolor('whitesmoke')
    ha = ax1.hist2d(
        data[0], data[1], 
        bins=[hillshade_range, bins],
        vmin=0,
        vmax=500,
        cmin=1,
    )
    PlotBase.insert_colorbar(ax1, ha[3], 'Count', right=0.9, rect=[0.91, 0.508, 0.03, 0.368])
    ax1.set_ylabel(SNOW_DEPTH_LABEL)
    ax1.set_title(title)

    data_ax2 = ax2.scatter(data[2], data[3], label='Median', s=2)
    ax2.set_ylabel(r'Median $\Delta m$')
    ax2.set_xlim(HILLSHADE_MIN, HILLSHADE_MAX)
    ax2.set_xlabel('Hillshade')

    ax2_2 = ax2.twinx()

    data_ax2_2 = ax2_2.scatter(data[4], data[5], label='Count', color='indigo', s=2)
    PlotBase.format_axes_scientific(ax2_2, 'y', (4, 4))
    
    data = [data_ax2, data_ax2_2]
    labels = [l.get_label() for l in data]
    ax2.legend(data, labels);

## Negative values in vegetation
### Snow Free scene

__0__ - Full shade

__255__ - No shade

In [ ]:
compare_column(negative_sfm[vegetation], 'hillshade_snow_free', 'Negative Snow Free Hillshade');

### Snow On

In [ ]:
compare_column(negative_sfm[vegetation], 'hillshade_snow_on', 'Negative Snow On Hillshade');

## Hillshade comparison (No Vegetation)
### Snow free

In [ ]:
compare_column(negative_sfm[open_areas], 'hillshade_snow_free', 'Negative Snow Free Hillshade');

### Snow On

In [ ]:
compare_column(negative_sfm[open_areas], 'hillshade_snow_on', 'Negative Snow On Hillshade');